# Template brains
Over the years, various groups have defined various brain templates: `FCWB` (FlyCircuit Wild Type Brain), `IS2`, `JRC2018F` or `JRC2018U` to name but a few. Some of these, like `JRC2018U` and `JRC2018F`, have been generated by averaging across multiple brains (Bogovic et al., 2018). Others, such as `FAFB` or `JRCFIB2018F` (Janelia hemibrain), represent single brains. 

For central adult fly neurons, VFB typically has data in `JRCFIB2018F` and sometimes additionally in a second template space.

# Transforms 
Transforms between two template brains come in various flavours and levels of sophistication: from simple landmark-based (thin plate spine) transforms to warp transforms using deformation fields (e.g. CMTK or Hdf5). 

Below bridging graph from [`navis-flybrains`](https://github.com/schlegelp/navis-flybrains) shows transforms that are currently available (to the best of my knowledge).

![](https://raw.githubusercontent.com/schlegelp/navis-flybrains/main/_static/bridging_graph.png)

Arrows indicate the forward direction but transforms can be performed both-ways. As you can see the graph is fully connected (`V2` and `VNCIS2` are VNC templates) meaning that we can transform data to any of the available brain spaces even if it sometimes means we need to chain multiply transforms. 

# The Example Use Case
Given a query neuron, you can ask VFB for morphologically similar neurons in a different dataset (see the nblast notebook). But there are situations where this is not so straight forward: what if you want a match in the Janelia hemibrain dataset but your query neuron is on the left side of the brain? It probably won't exist in the hemibrain dataset. We can work around this though! Here's a quick outline of what we will do:

1. Mirror our neuron from the left to the right hemisphere
2. Transform the mirrored neuron to hemibrain space
3. Fetch a couple of potential matches via neuprint 
4. Run an NBLAST

Note that we have already downloaded the transforms we need for this exercise (not all because they are quite big).

In [2]:
import os 
os.environ['FLYBRAINS_DATA']

'/root/work/training/transforms'

In [ ]:
import flybrains

In [ ]:
# Now we need to download the required transforms
# Note that we only download one set of transforms because it's sufficient for what we will be doing 
# Also the default directory these are downloaded to does not persist across DeepNote sessions 
# and we need to run this again every time the hardware resets 
# -> download takes ~3mins
import flybrains

flybrains.download_saalfeldlab_transforms()

Transforms:   0%|          | 0/6 [00:00<?, ?it/s]

JRC2018F_FAFB.h5:   0%|          | 0.00/580M [00:00<?, ?B/s]

JRC2018F_JFRC2013.h5:   0%|          | 0.00/1.39G [00:00<?, ?B/s]

JRC2018F_FCWB.h5:   0%|          | 0.00/1.29G [00:00<?, ?B/s]

JRC2018F_JRCFIB2018F.h5:   0%|          | 0.00/1.29G [00:00<?, ?B/s]

JRC2018U_JRC2018F.h5:   0%|          | 0.00/717M [00:00<?, ?B/s]

JRC2018F_JFRC2010.h5:   0%|          | 0.00/1.65G [00:00<?, ?B/s]

In [ ]:
# Import libs and initialise API objects
from vfb_connect.cross_server_tools import VfbConnect
import pandas as pd
vc = VfbConnect()

import pymaid
import navis

navis.set_pbars(jupyter=False)
pymaid.set_pbars(jupyter=False)

# Connect to the VFB CATMAID server hosting the FAFB data
rm = pymaid.connect_catmaid(server="https://fafb.catmaid.virtualflybrain.org/", api_token=None, max_threads=10)

# Test call to see if connection works 
print(f'Server is running CATMAID version {rm.catmaid_version}')


In [ ]:
tp = vc.neo_query_wrapper.get_templates()
[t['term']['core']['label'] for t in tp]

['JRC2018Unisex',
 'adult brain template JFRC2',
 'adult VNS template - Court2018',
 'adult brain template Ito2014',
 'JRC_FlyEM_Hemibrain',
 'JRC2018UnisexVNC',
 'L3 CNS template - Wood2018',
 'L1 larval CNS ssTEM - Cardona/Janelia']

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=94cb7c07-bfd7-4142-b5f9-bec41746df04' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>